In [123]:
import numpy as np
class cbuff():
    def __init__(self, l, step):
        self.buff = np.zeros(l, dtype=np.int32)
        self.buff.fill(-1)
        self.buff[0] = 0
        self.pos = 0
        self.len = 1
        self.step = step
        self.generate(l)
    
    def insert(self, pos, val):
        self.buff[pos+1:self.len+1] = self.buff[pos:self.len]
        self.buff[pos] = val
        self.len += 1
        
    def generate(self, n):
        for i in range(1, n):
            self.pos = ((self.pos + self.step) % self.len) + 1
            self.insert(self.pos, i)
            
    def index(self, i):
        idx, = np.where(self.buff == i)
        return idx[0]
    
    def __getitem__(self, args):
        return self.buff[args]
    
    def __repr__(self):
        return self.buff.__repr__()
    

class fake_cbuff(cbuff):
    def __init__(self, l, step, obs):
        self.obs = obs
        self.obsind = 0
        super().__init__(l, step)
        
    def insert(self, pos, val):
        if val <= self.obs:
            super().insert(pos, val)
            if val == self.obs:
                self.obsind = pos
        else:
            if pos <= self.obsind:
                self.buff[self.obsind:self.obsind+3] = self.buff[self.obsind-1:self.obsind+2]
                self.obsind += 1
            elif (pos == (self.obsind + 1)):
                self.buff[pos] = val
            self.len += 1
    
    

In [124]:
b = fake_cbuff(2018, 3, 2017)
idx, = np.where( b.buff == 2017 )
assert(b.buff[idx+1] == 638)

In [79]:
b = cbuff(10, 3)
assert(b.buff.tolist() == [0, 9, 5,  7,  2,  4,  3,  8,  6,  1])
b = cbuff(2018, 3)
idx, = np.where( b.buff == 2017 )
assert(b.buff[idx+1] == 638)

In [80]:
buff2017 = cbuff(2018, 3)
assert(buff2017.buff[buff2017.index(2017) + 1] == 638)

In [56]:
# part 1
inp = 359
b = cbuff(2018, inp)
index = b.index(2017)
print(b.buff[index+1])

1506


In [136]:
%%time
# part 2
inp = 359
buff = fake_cbuff(50000001, inp, 0)
idx = buff.index(0)
print(buff[idx+1])

39479736
CPU times: user 47.2 s, sys: 217 ms, total: 47.4 s
Wall time: 48.6 s


In [3]:
%%time
from numba import jit

@jit
def next0(step):
    pos = 0
    final = 0
    for i in range(1,50000001):
        pos = (pos+step)%i+1
        if pos==1:
            final = i
    return(final) 
print('Value after 0 in completed buffer is', next0(359))

Value after 0 in completed buffer is 39479736
CPU times: user 526 ms, sys: 4.54 ms, total: 531 ms
Wall time: 536 ms
